## Simple notebook that loads test data and applies the ensemble Kalman Filters.

We compare the mean of the analysis ensembles and the covariance matrices.
Actually only 1000 entries of the covariance matrix are computed due to the large size of the prior ensemble. Ther perturbations don't have to be equal

In [3]:
%cd kalmanfilters

[Errno 2] No such file or directory: 'kalmanfilters'
/home/ldap-server/draco/notebooks/kalmanfilters


In [37]:
from ensrf import *
from ensrf_direct import *
from estkf import *
from etkf import *
from etkf_livings import *
from ensrf_serial import *
import numpy as np
import scipy
from time import time

In [33]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
#You can look into the imported files using

%less estkf.py

In [63]:
###LOAD TEST DATA

Y=np.load('../testdata/Y.npy',allow_pickle=True)
R=np.load('../testdata/R.npy',allow_pickle=True)
Xf=np.load('../testdata/Xf.npz',allow_pickle=True)['arr_0']
HXf=np.load('../testdata/HXf.npy',allow_pickle=True)

print('Y shape:',np.shape(Y))
print('R shape:',np.shape(R))
print('Xf shape:',np.shape(Xf))
print('HXf shape:',np.shape(HXf))

Y shape: (293,)
R shape: (293,)
Xf shape: (55296, 100)
HXf shape: (293, 100)


In [58]:
import numpy as np

variables=[Xf, HXf, Y, R]

mean={}
cov={}

funcs=[ESTKF, EnSRF, EnSRF_serial,ENSRF_direct, ETKF, ETKF_livings]

for i,f in enumerate(funcs):
    name=str(f.__name__)
    begin = time()
    full=f(*variables)
    end=time ()
    print('-------------------------------')
    print(name,' executed in ',end-begin, 'seconds')
    print('-------------------------------')
    mean[name]=np.mean(full,axis=1)
    cov[name]=np.cov(full[:1000,:],ddof=1)

-------------------------------
ESTKF  executed in  0.056397438049316406 seconds
-------------------------------
-------------------------------
EnSRF  executed in  0.1246793270111084 seconds
-------------------------------
-------------------------------
EnSRF_serial  executed in  10.726128101348877 seconds
-------------------------------
-------------------------------
ENSRF_direct  executed in  0.41527724266052246 seconds
-------------------------------
-------------------------------
ETKF  executed in  0.05635523796081543 seconds
-------------------------------
-------------------------------
ETKF_livings  executed in  0.06243419647216797 seconds
-------------------------------


In [55]:
mean

{'ESTKF': array([225.77951468, 225.75510173, 225.75508779, ..., 252.08298075,
        252.08490147, 252.08658989]),
 'EnSRF': array([225.77951468+0.j, 225.75510173+0.j, 225.75508779+0.j, ...,
        252.08298075+0.j, 252.08490147+0.j, 252.08658989+0.j]),
 'EnSRF_serial': array([225.77951468, 225.75510173, 225.75508779, ..., 252.08298075,
        252.08490147, 252.08658989]),
 'ENSRF_direct': array([225.77951468+1.02921696e-32j, 225.75510173+9.65516829e-33j,
        225.75508779+1.08391337e-32j, ..., 252.08298075+5.84616035e-33j,
        252.08490147+1.06922816e-32j, 252.08658989+1.66809607e-32j]),
 'ETKF': array([225.77951468, 225.75510173, 225.75508779, ..., 252.08298075,
        252.08490147, 252.08658989])}

In [62]:
#check equality of means
m=list(mean.values()) 
for i,me in enumerate(m):
    if i<len(m)-1:
        print(np.allclose(m[i],m[i+1]))

True
True
True
True
True


In [61]:
#check equality of covariance matrix entries
c=list(cov.values()) 
for i,me in enumerate(c):
    if i<len(c)-1:
        print(np.allclose(c[i],c[i+1]))

True
True
True
True
True
